In [0]:
from pyspark.sql.types import*
mySchema =  StructType ([StructField("index", IntegerType()) ,
                         StructField("webpage", StringType()) ,
                         StructField("associated_files", StringType())])
                         
myRDD= sc.textFile("/FileStore/webpage/*")\
                         .map(lambda line: line.split("\t"))\
                         .map(lambda values:[int(values[0]), values[1], values[2]])

webpages= spark.createDataFrame(myRDD, mySchema)   
                         
webpages.show(truncate=False) 

+-----+-------------------------------+------------------------------------------+
|index|webpage                        |associated_files                          |
+-----+-------------------------------+------------------------------------------+
|1    |sorrento_f00l_sales.html       |theme.css,code.js,sorrento_f00l.jpg       |
|2    |titanic_2100_sales.html        |theme.css,code.js,titanic_2100.jpg        |
|3    |meetoo_3.0_sales.html          |theme.css,code.js,meetoo_3.0.jpg          |
|4    |meetoo_3.1_sales.html          |theme.css,code.js,meetoo_3.1.jpg          |
|5    |ifruit_1_sales.html            |theme.css,code.js,ifruit_1.jpg            |
|6    |ifruit_3_sales.html            |theme.css,code.js,ifruit_3.jpg            |
|7    |ifruit_2_sales.html            |theme.css,code.js,ifruit_2.jpg            |
|8    |ifruit_5_sales.html            |theme.css,code.js,ifruit_5.jpg            |
|9    |titanic_1000_sales.html        |theme.css,code.js,titanic_1000.jpg        |
|10 

In [0]:
webpages.createOrReplaceTempView("Views")

In [0]:
spark.sql("CREATE OR REPLACE TEMP VIEW s_views AS SELECT * FROM Views WHERE webpage LIKE 's%'")

Out[94]: DataFrame[]

In [0]:
createDF= spark.sql("SELECT * FROM Views WHERE webpage LIKE 's%'")

In [0]:
spark.sql("CREATE OR REPLACE TEMP VIEW sub_views AS SELECT webpage, associated_files FROM Views")

Out[96]: DataFrame[]

In [0]:
spark.catalog.listTables()

Out[97]: [Table(name='explode', database=None, description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='exploded_views', database=None, description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='s_views', database=None, description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='spliteg', database=None, description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='splitviews', database=None, description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='sub_views', database=None, description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='views', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

In [0]:
spark.sql("SELECT * FROM views").head(5)

Out[98]: [Row(index=1, webpage='sorrento_f00l_sales.html', associated_files='theme.css,code.js,sorrento_f00l.jpg'),
 Row(index=2, webpage='titanic_2100_sales.html', associated_files='theme.css,code.js,titanic_2100.jpg'),
 Row(index=3, webpage='meetoo_3.0_sales.html', associated_files='theme.css,code.js,meetoo_3.0.jpg'),
 Row(index=4, webpage='meetoo_3.1_sales.html', associated_files='theme.css,code.js,meetoo_3.1.jpg'),
 Row(index=5, webpage='ifruit_1_sales.html', associated_files='theme.css,code.js,ifruit_1.jpg')]

In [0]:
spark .sql(" CREATE OR REPLACE TEMP VIEW spliteg AS SELECT split ( webpage , '_') as split_wp , associated_files FROM views ")

Out[99]: DataFrame[]

In [0]:
spark.sql("SELECT * FROM spliteg").head(5)

Out[100]: [Row(split_wp=['sorrento', 'f00l', 'sales.html'], associated_files='theme.css,code.js,sorrento_f00l.jpg'),
 Row(split_wp=['titanic', '2100', 'sales.html'], associated_files='theme.css,code.js,titanic_2100.jpg'),
 Row(split_wp=['meetoo', '3.0', 'sales.html'], associated_files='theme.css,code.js,meetoo_3.0.jpg'),
 Row(split_wp=['meetoo', '3.1', 'sales.html'], associated_files='theme.css,code.js,meetoo_3.1.jpg'),
 Row(split_wp=['ifruit', '1', 'sales.html'], associated_files='theme.css,code.js,ifruit_1.jpg')]

In [0]:
spark.sql(" CREATE OR REPLACE TEMP VIEW splitviews AS SELECT split ( associated_files , ',') as split_af, webpage FROM views ")

Out[101]: DataFrame[]

In [0]:
spark.sql("SELECT * FROM views").head(5)

Out[102]: [Row(index=1, webpage='sorrento_f00l_sales.html', associated_files='theme.css,code.js,sorrento_f00l.jpg'),
 Row(index=2, webpage='titanic_2100_sales.html', associated_files='theme.css,code.js,titanic_2100.jpg'),
 Row(index=3, webpage='meetoo_3.0_sales.html', associated_files='theme.css,code.js,meetoo_3.0.jpg'),
 Row(index=4, webpage='meetoo_3.1_sales.html', associated_files='theme.css,code.js,meetoo_3.1.jpg'),
 Row(index=5, webpage='ifruit_1_sales.html', associated_files='theme.css,code.js,ifruit_1.jpg')]

In [0]:
spark.sql("SELECT * FROM splitviews").head(5)

Out[103]: [Row(split_af=['theme.css', 'code.js', 'sorrento_f00l.jpg'], webpage='sorrento_f00l_sales.html'),
 Row(split_af=['theme.css', 'code.js', 'titanic_2100.jpg'], webpage='titanic_2100_sales.html'),
 Row(split_af=['theme.css', 'code.js', 'meetoo_3.0.jpg'], webpage='meetoo_3.0_sales.html'),
 Row(split_af=['theme.css', 'code.js', 'meetoo_3.1.jpg'], webpage='meetoo_3.1_sales.html'),
 Row(split_af=['theme.css', 'code.js', 'ifruit_1.jpg'], webpage='ifruit_1_sales.html')]

In [0]:
spark.sql("CREATE OR REPLACE TEMP VIEW exploded_views AS SELECT webpage, explode(split_af) FROM splitviews")

Out[104]: DataFrame[]

In [0]:
spark.sql("SELECT * FROM exploded_views").head(5)

Out[105]: [Row(webpage='sorrento_f00l_sales.html', col='theme.css'),
 Row(webpage='sorrento_f00l_sales.html', col='code.js'),
 Row(webpage='sorrento_f00l_sales.html', col='sorrento_f00l.jpg'),
 Row(webpage='titanic_2100_sales.html', col='theme.css'),
 Row(webpage='titanic_2100_sales.html', col='code.js')]

In [0]:
from pyspark .sql . functions import split
splitDF= webpages.withColumn ('splitWebpage', split ( webpages ["webpage"] , "_"))
splitDF.head(5)

Out[117]: [Row(index=1, webpage='sorrento_f00l_sales.html', associated_files='theme.css,code.js,sorrento_f00l.jpg', splitWebpage=['sorrento', 'f00l', 'sales.html']),
 Row(index=2, webpage='titanic_2100_sales.html', associated_files='theme.css,code.js,titanic_2100.jpg', splitWebpage=['titanic', '2100', 'sales.html']),
 Row(index=3, webpage='meetoo_3.0_sales.html', associated_files='theme.css,code.js,meetoo_3.0.jpg', splitWebpage=['meetoo', '3.0', 'sales.html']),
 Row(index=4, webpage='meetoo_3.1_sales.html', associated_files='theme.css,code.js,meetoo_3.1.jpg', splitWebpage=['meetoo', '3.1', 'sales.html']),
 Row(index=5, webpage='ifruit_1_sales.html', associated_files='theme.css,code.js,ifruit_1.jpg', splitWebpage=['ifruit', '1', 'sales.html'])]

In [0]:
from pyspark.sql.functions import split
splitDF2= webpages.withColumn('splitaf', split( webpages ["associated_files"], ","))
splitDF2.head(5)

Out[113]: [Row(index=1, webpage='sorrento_f00l_sales.html', associated_files='theme.css,code.js,sorrento_f00l.jpg', splitaf=['theme.css', 'code.js', 'sorrento_f00l.jpg']),
 Row(index=2, webpage='titanic_2100_sales.html', associated_files='theme.css,code.js,titanic_2100.jpg', splitaf=['theme.css', 'code.js', 'titanic_2100.jpg']),
 Row(index=3, webpage='meetoo_3.0_sales.html', associated_files='theme.css,code.js,meetoo_3.0.jpg', splitaf=['theme.css', 'code.js', 'meetoo_3.0.jpg']),
 Row(index=4, webpage='meetoo_3.1_sales.html', associated_files='theme.css,code.js,meetoo_3.1.jpg', splitaf=['theme.css', 'code.js', 'meetoo_3.1.jpg']),
 Row(index=5, webpage='ifruit_1_sales.html', associated_files='theme.css,code.js,ifruit_1.jpg', splitaf=['theme.css', 'code.js', 'ifruit_1.jpg'])]

In [0]:
from pyspark.sql.functions import explode
explodedDF = splitDF.withColumn('explodedWebpage', explode(splitDF.splitWebpage))
explodedDF.head(5)

Out[118]: [Row(index=1, webpage='sorrento_f00l_sales.html', associated_files='theme.css,code.js,sorrento_f00l.jpg', splitWebpage=['sorrento', 'f00l', 'sales.html'], explodedWebpage='sorrento'),
 Row(index=1, webpage='sorrento_f00l_sales.html', associated_files='theme.css,code.js,sorrento_f00l.jpg', splitWebpage=['sorrento', 'f00l', 'sales.html'], explodedWebpage='f00l'),
 Row(index=1, webpage='sorrento_f00l_sales.html', associated_files='theme.css,code.js,sorrento_f00l.jpg', splitWebpage=['sorrento', 'f00l', 'sales.html'], explodedWebpage='sales.html'),
 Row(index=2, webpage='titanic_2100_sales.html', associated_files='theme.css,code.js,titanic_2100.jpg', splitWebpage=['titanic', '2100', 'sales.html'], explodedWebpage='titanic'),
 Row(index=2, webpage='titanic_2100_sales.html', associated_files='theme.css,code.js,titanic_2100.jpg', splitWebpage=['titanic', '2100', 'sales.html'], explodedWebpage='2100')]

In [0]:
explodedDF = splitDF2.withColumn('explodedaf', explode(splitDF2.splitaf))
explodedDF.head(5)

Out[116]: [Row(index=1, webpage='sorrento_f00l_sales.html', associated_files='theme.css,code.js,sorrento_f00l.jpg', splitaf=['theme.css', 'code.js', 'sorrento_f00l.jpg'], explodedaf='theme.css'),
 Row(index=1, webpage='sorrento_f00l_sales.html', associated_files='theme.css,code.js,sorrento_f00l.jpg', splitaf=['theme.css', 'code.js', 'sorrento_f00l.jpg'], explodedaf='code.js'),
 Row(index=1, webpage='sorrento_f00l_sales.html', associated_files='theme.css,code.js,sorrento_f00l.jpg', splitaf=['theme.css', 'code.js', 'sorrento_f00l.jpg'], explodedaf='sorrento_f00l.jpg'),
 Row(index=2, webpage='titanic_2100_sales.html', associated_files='theme.css,code.js,titanic_2100.jpg', splitaf=['theme.css', 'code.js', 'titanic_2100.jpg'], explodedaf='theme.css'),
 Row(index=2, webpage='titanic_2100_sales.html', associated_files='theme.css,code.js,titanic_2100.jpg', splitaf=['theme.css', 'code.js', 'titanic_2100.jpg'], explodedaf='code.js')]